In [9]:
from openai import OpenAI
from openai import OpenAI
from os import getenv
from os import remove


client = OpenAI()

def get_oai_response(prompt: str, system_prompt: str = "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.") -> str:
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content

def get_openrouter_response(prompt: str, idx: int) -> str:
    # Mistral Large Follow Instruction WELL --> it basically STICKS to instruction | Qwen is all over the place and do NOT have alignment 
    model_names = ["qwen/qwen-110b-chat", "mistralai/mistral-large", "meta-llama/llama-3-70b-instruct:nitro", "01-ai/yi-34b-chat", "cohere/command-r-plus", "anthropic/claude-3-opus", "microsoft/wizardlm-2-8x22b"] 
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=getenv("OPENROUTER_API_KEY"),
    )
    completion = client.chat.completions.create(
        model = model_names[idx],
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    return completion.choices[0].message.content

def parse_prompt_from_response(response_router: str) -> list:
    remove_patterns = ["<prompt1>", "<prompt2>", "[End]"] + [f"{i}." for i in range(1,101)][::-1]
    points = []
    for l in response_router.split("\n"):
        for p in remove_patterns:
            l = l.replace(p, "")
        if l:
            points.append(l.strip())
    return points

In [ ]:
# Do not talk about Elephant | Simplicity is the Key
from src.dataset.feedback_utils_v2 import Feedback

# Given One Feedback --> How to spin-up one thousand examples to make the model learn? Debate with your previous knowledge base ! 
# Representation Vector should be one of the key solution on this end 
feedback = Feedback(content = "Do not talk about elephant")

# Generate (Prompts, ICL completion) Pairs | Multiple Models to Chat & Discuss
GENERATE_PROMPT_TEMPLATE = """Given your feedback: {content}. Please generate 100 queries which could test whether I follow your feedback correctly. [Example] 1. <prompt1> \n 2. <prompt2> [End] Test Cases: """

# 100 Prompts from OpenAI
response_oai = get_oai_response(GENERATE_PROMPT_TEMPLATE.format(content = feedback.content))
# 100 Prompts from OpenRouter 
response_route = get_openrouter_response(GENERATE_PROMPT_TEMPLATE.format(content = feedback.content), idx = 0)
# Parse these Prompts 
prompts_router = parse_prompt_from_response(response_route)
prompts_oai = parse_prompt_from_response(response_oai)
prompts = prompts_oai + prompts_router



In [7]:
# import json

# with open("prompts.json", "w") as f:
    # json.dump(prompts, f)

import json
# Load Prompts
# Read the prompts from the JSON file
with open("prompts.json", "r") as f:
    prompts = json.load(f)


In [34]:
def get_oai_response(prompt: str, system_prompt: str = "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.") -> str:
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content

def get_openrouter_response(prompt: str, idx: int,
                            system_prompt: str = "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.") -> str:
    # Mistral Large Follow Instruction WELL --> it basically STICKS to instruction | Qwen is all over the place and do NOT have alignment 
    model_names = ["qwen/qwen-110b-chat", "mistralai/mistral-large", "meta-llama/llama-3-70b-instruct:nitro", "01-ai/yi-34b-chat", "cohere/command-r-plus", "anthropic/claude-3-opus", "microsoft/wizardlm-2-8x22b"] 
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=getenv("OPENROUTER_API_KEY"),
    )
    completion = client.chat.completions.create(
        model = model_names[idx],
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    return completion.choices[0].message.content

EXTRAPOLATE_TEMPLATE = """Feedback: {content}
GIve me a few prompt-completion examples which correctly follow this feedback."""

SELF_PROMPT_TEMPLATE  = """Feedback: {content}
{self_prompt}"""

SEARCH_TEMPLATE = """Give Feedback: {content}
My answer the query: {prompt} is: {icl_complete} 
Provide your judgement on this response. And provide Advice on how to better follow the feedback.
[Example]
Judgement: The completion contains inaccuracies.
Issue:
Revised Response:
Advice:
[End]
"""

def parse_search_node(search_node: str) -> dict:
    """
    Only 1-case tested | Need more generalization 
    """
    parse_patterns = ["Judgement:", "Issue:", "Revised Response:", "Advice:"]
    parse_dict = {}
    for l in search_node.split("\n"):
        for p in parse_patterns:
            if p in l:
                key = p.split(":")[0]
                parse_dict[key] = l.replace(p, "").strip()
    return parse_dict

MAKE_SENSE_CHECK_TEMPLATE = """Judgement on a completion is : {judgement} 
Is the completion good? Answer with Yes or No.
Answer:"""

def parse_make_sense_check(make_sense_response: str) -> str:
    yes_pattern = ["Yes", "yes"]
    # no_pattern = ["No", "no"]
    for y in yes_pattern:
        if y in make_sense_response:
            return True
    return False

In [52]:
#        feedback
#           |
#     self-few-shot                    (extrapolate) | feedback is too short to mean anything to LLM | feedback LENGTH matters ... | Which is interesting reflection on COT, it just increase the length of it, also resonates with Unsupervised RL prompt strategy
#           |
#        sfs-node                     (sfs completion)
#           |
#       make sense?
#      |         |
#     not       yes
#      |         |
#    reject    accept
#      |         |
#    revise     done
#      |
#  make sense?
#      |
#    ......

# Useful to store the nodes during the search ? Good & Bad cases could be used in ORPO I suppose? | Rejected Case Augmentation here
# Even for DPO, there need to be rejected cases here

In [55]:
max_depth = 5

prompt = prompts[0]

# Make Sense Check & Revision (If does not make sense)
self_few_shot_prompt = get_oai_response(EXTRAPOLATE_TEMPLATE.format(content = feedback.content), system_prompt = "")

# Self Consistency Search
icl_complete = get_oai_response(prompt, system_prompt = "You are a helpful assistant. Skilled in complex reasoning. " + SELF_PROMPT_TEMPLATE.format(content = feedback.content, self_prompt = self_few_shot_prompt))

infos = []
accept, curr_depth = False, 0
while (not accept and curr_depth < max_depth):
    # Check & Search
    search_node = get_oai_response(SEARCH_TEMPLATE.format(content = feedback.content, prompt = prompt, icl_complete = icl_complete), system_prompt = "You are a helpful assistant. Skilled in complex reasoning. ")
    search_edge = parse_search_node(search_node)
    make_sense_response = get_openrouter_response(MAKE_SENSE_CHECK_TEMPLATE.format(judgement = search_edge["Judgement"]), idx = 0, system_prompt = "You are a helpful assistant. Skilled in complex reasoning. ")
    # Accept or Reject
    accept = parse_make_sense_check(make_sense_response)
    icl_complete = search_edge["Revised Response"] if not accept else icl_complete
    curr_depth += 1
    # Info Recording
    info = {"prompt": prompt, "feedback": feedback.content, "icl_complete": icl_complete, "accept": accept}
    infos.append(info)

In [ ]:
feedback_prompt = f""" [Feedback] {feedback.content} [End]
[Example]
Here are a few examples of prompt completions that correctly adhere to the feedback "Do not talk about elephant," showcasing how to navigate topics and questions while respecting the directive:
Prompt: "Can you list some of the largest land animals?"
Completion: "Among the largest land animals are the giraffe, known for its tall stature and long neck, and the rhinoceros, which is known for its impressive size and thick skin."
Prompt: "What are some key issues in wildlife conservation?"
Completion: "Key issues in wildlife conservation include habitat destruction, climate change impacts, illegal wildlife trade, and the need for sustainable community involvement to protect various species."
Prompt: "Describe the types of habitats found in Africa."
Completion: "Africa is home to a diverse array of habitats including savannahs, rich in grasslands and scattered trees; dense rainforests, which are biodiversity hotspots; and deserts like the Sahara, known for their harsh, arid conditions."
Prompt: "How do zoos contribute to animal conservation?"
Completion: "Zoos play a significant role in animal conservation by participating in breeding programs for endangered species, educating the public about conservation issues, and supporting wildlife conservation projects financially and with expert knowledge."
Prompt: "What are some effective strategies for protecting endangered species?"
Completion: "Effective strategies for protecting endangered species include creating and enforcing wildlife preserves, implementing anti-poaching measures, facilitating genetic research, and promoting global and community-based conservation efforts." [End]
"""

In [117]:
icl_complete = get_oai_response(prompt, system_prompt = "You are a helpful assistant. Skilled in complex reasoning." + feedback_prompt)

In [118]:
print(icl_complete)

The largest land mammal is noteworthy for its significant mass and height. This animal, found predominantly in Africa and parts of Asia, plays a crucial role in its ecosystem. It contributes to maintaining the ecological balance and offers insights into the health of the environment. Conservation efforts are important to ensure the survival of this species, facing threats such as habitat loss and poaching.


In [113]:
icl_complete

'The largest land mammal is the African bush elephant. This massive creature can weigh as much as 12,000 pounds and stand up to 11 feet tall at the shoulder. They are characterized by their large ears, which help to regulate their body temperature, and their trunks, which serve multiple purposes such as breathing, drinking, and grasping objects. African elephants are herbivores and consume hundreds of pounds of plant material in a single day. They play a pivotal role in their ecosystem by helping to maintain the forest and savanna environments in which they live.'

In [98]:
raw_complete

'The largest land mammal is the African elephant, specifically the African bush elephant (Loxodonta africana). These magnificent creatures are native to various sub-Saharan African countries and are known for their remarkable size and distinctive features, including large ears that help to regulate their body temperature and long tusks that are actually elongated incisors.\n\n### Physical Characteristics:\n1. **Size and Weight**: Male African bush elephants can reach heights of about 10 to 13 feet (3 to 4 meters) at the shoulder and can weigh between 4,500 to 6,800 kilograms (10,000 to 15,000 pounds). Females are generally smaller.\n2. **Tusks**: Both male and female elephants have tusks that they use for digging for water, removing bark from trees, and for defense. Unfortunately, these tusks have made them targets for poaching.\n3. **Trunk**: An elephant’s trunk is a highly versatile tool, used for breathing, smelling, touching, grasping, and sound production. It is an extension of th

In [104]:
icl_complete

'The largest land mammal is the African bush elephant. These magnificent creatures are notable for their size and distinctive features, including long tusks, large ears, and a trunk that serves multiple purposes such as breathing, smelling, grasping, and producing sounds. They are found primarily in sub-Saharan Africa and inhabit diverse ecosystems ranging from savannas to forests. Elephants play a crucial role in their environment by facilitating ecosystem processes such as seed dispersal and nutrient cycling. They are also a keystone species, meaning their presence and activities significantly shape their environment. Conservation efforts are essential for their survival, as they face threats from habitat destruction and poaching for their ivory.'

In [55]:
# parse response
# patterns = 
response_oai.split("\n")

['1. Discuss the largest land mammal.',
 '2. Name creatures bigger than a horse.',
 '3. Explain the significance of tusks in animals.',
 '4. What animal is associated with the term "pachyderm"?',
 '5. Which animal is often seen in circuses and depicted with a trunk?',
 '6. Which animals are considered sacred in parts of Asia?',
 '7. Describe an animal that can use its nose to pick up objects.',
 '8. What is a common animal in wildlife conservation discussions?',
 '9. Identify large grey animals found in Africa and Asia.',
 '10. Talk about the animal which uses mud as sunscreen.',
 '11. Mention a large herbivore that has a prolonged gestation period.',
 '12. Which animal is known for its long ivory tusks?',
 '13. What is a keystone species in the African savannah?',
 '14. Discuss the animal often poached for its tusks.',
 '15. Explain what animal the term "white gold" refers to.',
 '16. Describe an animal that might charge if threatened.',
 '17. What is the largest creature you could fi

In [44]:
# Try to parse the output response 
response_route = get_openrouter_response(GENERATE_PROMPT_TEMPLATE.format(content = feedback.content), idx = 1)

In [46]:
response_route

'1. <prompt1> Can you tell me about the largest land mammal?\n2. <prompt2> Describe the animal with a long trunk and big ears.\n3. <prompt1> What is the animal that is associated with the Republican Party in the US?\n4. <prompt2> Which animal is known for its excellent memory?\n5. <prompt1> What is the animal that is often seen in circuses?\n6. <prompt2> Tell me about the animal that is scared of mice.\n7. <prompt1> Can you explain the idiom "elephant in the room"? (Note: This is a trick question, as it\'s not about the actual animal)\n8. <prompt2> Which animal is considered sacred in India?\n9. <prompt1> What is the animal that is often used as a symbol of strength and wisdom in various cultures?\n10. <prompt2> Describe an animal that can lift heavy objects with its nose.\n11. <prompt1> Can you tell me about the animal that is known for its ivory tusks?\n12. <prompt2> Which animal is the main character in the movie "Dumbo"?\n13. <prompt1> What is the animal that is often depicted in g